# Stock Trades by Members of the US House of Representatives

This project uses public data about the stock trades made by members of the US House of Representatives. This data is collected and maintained by Timothy Carambat as part of the [House Stock Watcher](https://housestockwatcher.com/) project. The project describes itself as follows:

> With recent and ongoing investigations of incumbent congressional members being investigated for potentially violating the STOCK act. This website compiles this publicly available information in a format that is easier to digest then the original PDF source.
>
> Members of Congress must report periodic reports of their asset transactions. This website is purely for an informative purpose and aid in transparency.
>
> This site does not manipluate or censor any of the information from the original source. All data is transcribed by our community of contributors, which you can join for free by going to our transcription tool. Our moderation team takes great care in ensuring the accuracy of the information.
>
> This site is built and maintained by Timothy Carambat and supported with our contributors.

Some interesting questions to consider for this data set include:

- Is there a difference in stock trading behavior between political parties? For example:
    - does one party trade more often?
    - does one party make larger trades?
    - do the two parties invest in different stocks or sectors? For instance, do Democrats invest in Tesla more than Republicans?
- What congresspeople have made the most trades?
- What companies are most traded by congresspeople?
- Is there evidence of insider trading? For example, Boeing stock dropped sharply in February 2020. Were there a suspiciously-high number of sales of Boeing before the drop?
- When are stocks bought and sold? Is there a day of the week that is most common? Or a month of the year?

### Getting the Data

The full data set of stock trade disclosures is available as a CSV or as JSON at https://housestockwatcher.com/api.

This data set does not, however, contain the political affiliation of the congresspeople. If you wish to investigate a question that relies on having this information, you'll need to find another dataset that contains it and perform a merge. *Hint*: Kaggle is a useful source of data sets.


### Cleaning and EDA

- Clean the data.
    - Certain fields have "missing" data that isn't labeled as missing. For example, there are fields with the value "--." Do some exploration to find those values and convert them to null values.
    - You may also want to clean up the date columns to enable time-series exploration.
- Understand the data in ways relevant to your question using univariate and bivariate analysis of the data as well as aggregations.


### Assessment of Missingness

- Assess the missingness per the requirements in `project03.ipynb`

### Hypothesis Test / Permutation Test
Find a hypothesis test or permutation test to perform. You can use the questions at the top of the notebook for inspiration.

# Summary of Findings

### Introduction
TODO

### Cleaning and EDA
TODO

### Assessment of Missingness
TODO

### Hypothesis Test
TODO

# Code

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import missingno as msno
import bs4 
import requests
import scrape_congress
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

### Cleaning and EDA

In [2]:
fp = os.path.join('data/', 'all_transactions.csv')
all_transactions = pd.read_csv(fp)
all_transactions.isna().sum()

disclosure_year              0
disclosure_date              0
transaction_date             0
owner                     4529
ticker                       0
asset_description            4
type                         0
amount                       0
representative               0
district                     0
ptr_link                     0
cap_gains_over_200_usd       0
dtype: int64

So let us begin by finding all potential fake missingness and turning it into real missingness. It appears, after some rigorous analysis (looking at each series potential values), that the only fake missingness exists in the 'owner' and 'ticker' columns. We will now replace those values with NaNs. In summary:

Missingness in data:
- Owner column has fake missing ('--') and real missing
- Ticker column has fake missing ('--')
- Asset description column has real missing but only 4 so we can hand clean this

In [3]:
all_transactions.iloc[732] #Replace NaN in a_d with Ball Corporation
all_transactions.iloc[3262] #Replace NaN in a_d with CELO
all_transactions.iloc[10537] #Replace NaN in a_d with URGO
all_transactions.iloc[10538] #Replace NaN in a_d with URGO

disclosure_year                                                        2021
disclosure_date                                                  03/18/2021
transaction_date                                                 2021-02-18
owner                                                                   NaN
ticker                                                                 URGO
asset_description                                                       NaN
type                                                               purchase
amount                                                     $1,001 - $15,000
representative                                              Hon. Brian Mast
district                                                               FL18
ptr_link                  https://disclosures-clerk.house.gov/public_dis...
cap_gains_over_200_usd                                                False
Name: 10538, dtype: object

In [4]:
all_transactions.at[732, 'asset_description'] = 'Ball Corporation'
all_transactions.at[3262, 'asset_description'] = 'CELO'
all_transactions.at[10537, 'asset_description'] = 'URGO'
all_transactions.at[10538, 'asset_description'] = 'URGO'

Our current strategy for tickers is to ignore the ticker and instead use the asset description wherever we can. The ticker is currently hypothesized to be NMAR after some brief exploration of some missing tickers: Missing dependent upon if the asset is listed on the NASDAQ. This can be further proven by understanding that the collection process populates the form according to NASDAQ tickers. 

In the following cell we sorted the transaction date values to find the smallest values from a string perspective, and discovered several anomalies. For these anomalies our intention is to hand clean the data and make sure that the date listed matches the date reported in the PDF sent in to Congress. 

In [5]:
all_transactions['transaction_date'].sort_values().head(10)

2069    0009-06-09
9381    0021-06-22
3496    0021-08-02
9382    0201-06-22
8662    2012-06-19
3489    2018-09-08
3488    2018-09-09
9627    2018-12-27
7875    2019-01-09
7535    2019-01-09
Name: transaction_date, dtype: object

In [6]:
all_transactions.iloc[8662] #Correct transaction date is: 06/19/2020
all_transactions.iloc[2069] #Correct transaction date is: 06/09/2021
all_transactions.iloc[9381] #Correct transaction date is: 06/22/2021
all_transactions.iloc[3496] #Correct transaction date is: 08/02/2021
all_transactions.iloc[9382] #Correct transaction date is: 06/22/2021

disclosure_year                                                        2021
disclosure_date                                                  07/16/2021
transaction_date                                                 0201-06-22
owner                                                                   NaN
ticker                                                                   KR
asset_description                                            Kroger Company
type                                                           sale_partial
amount                                                     $1,001 - $15,000
representative                                       Hon. James E Hon Banks
district                                                               IN03
ptr_link                  https://disclosures-clerk.house.gov/public_dis...
cap_gains_over_200_usd                                                False
Name: 9382, dtype: object

In [7]:
all_transactions.at[8662, 'transaction_date'] = '2020-06-19'
all_transactions.at[2069, 'transaction_date'] = '2021-06-09'
all_transactions.at[9381, 'transaction_date'] = '2021-06-22'
all_transactions.at[3496, 'transaction_date'] = '2021-08-02'
all_transactions.at[9382, 'transaction_date'] = '2021-06-22'

In [8]:
all_transactions['transaction_date'].sort_values().head(10)

3489    2018-09-08
3488    2018-09-09
9627    2018-12-27
7475    2019-01-09
7535    2019-01-09
7797    2019-01-09
7469    2019-01-09
7560    2019-01-09
7420    2019-01-09
7570    2019-01-09
Name: transaction_date, dtype: object

In [9]:
transactions_w_nan = all_transactions.replace('--', np.nan)

In [10]:
transactions_w_nan.isna().sum()

disclosure_year              0
disclosure_date              0
transaction_date             0
owner                     5844
ticker                    1023
asset_description            0
type                         0
amount                       0
representative               0
district                     0
ptr_link                     0
cap_gains_over_200_usd       0
dtype: int64

In [11]:
transactions_w_nan['transaction_date'] = pd.to_datetime(transactions_w_nan['transaction_date'])
transactions_w_nan['transaction_date'].dtypes

dtype('<M8[ns]')

In [12]:
transactions_w_nan['disclosure_date'] = pd.to_datetime(transactions_w_nan['disclosure_date'])
transactions_w_nan['disclosure_date'].dtypes

dtype('<M8[ns]')

Now we need to join our dataset with the parties for each district DURING THE TIME PERIOD OF EACH DISCLOSURE. After some exploration of the official roster on the house page, we discovered that some representatives in this dataset are no longer a a part of the house. Thus our only recourse is to scrape the historical list of house members provided on this webpage: https://www.congress.gov/members?pageSize=250&q=%7B%22chamber%22%3A%22House%22%2C%22congress%22%3A%5B%22116%22%2C117%5D%7D and match entries using the district name, and date.

If possible during this process, we'd like to clean names as well. This is because during our initial exploration we discovered a single glaring naming inconsistency:

In [62]:
transactions_w_nan[transactions_w_nan['representative'].str.contains('Dunn')]['representative'].unique()

array(['Hon. Neal Patrick MD, Facs Dunn',
       'Hon. Neal Patrick MD, FACS Dunn',
       'Hon. Neal Patrick Dunn MD, FACS'], dtype=object)

So, let us scrape that webpage now. After a pair programming session we moved all of that work into a helper that takes in the webpage we're interested in:

In [14]:
historical_members_1 = scrape_congress.scrape_congress_page('https://www.congress.gov/members?pageSize=250&q=%7B%22chamber%22%3A%22House%22%2C%22congress%22%3A%5B%22116%22%2C117%5D%7D')
historical_members_2 = scrape_congress.scrape_congress_page('https://www.congress.gov/members?q=%7B%22chamber%22%3A%22House%22%2C%22congress%22%3A%5B%22116%22%2C%22117%22%5D%7D&pageSize=250&page=2')
historical_members_3 = scrape_congress.scrape_congress_page('https://www.congress.gov/members?q=%7B%22chamber%22%3A%22House%22%2C%22congress%22%3A%5B%22116%22%2C%22117%22%5D%7D&pageSize=250&page=3')

In the next cell, we use webscraping to produce a dictionary of state names and abbreviations. This is so that we can add district codes to our new dataset of official congresspeople names.

In [15]:
site = requests.get('https://abbreviations.yourdictionary.com/articles/state-abbrev.html')
site.status_code
soup = bs4.BeautifulSoup(site.text, features = 'html.parser')
abbr_dct = {name.find_all('td')[0].text: name.find_all('td')[1].text for name in soup.find_all('tr')[1:]}

There were several strategies discussed for adding parties to our dataset. The main strategy we considered was merging on date and district code. After seeing how little existed in terms of dates, and district codes as far as party allegiance was concerned, we decided to change tracks and opted for a more cohesive strategy. Essentially we will join the dataframes by creating a unique identifier column that will contain the last name and the district code for each member of congress. This works because each district will have only one representative, and it is highly unlikely for a district to have two representatives with the same last name in a row. In fact, we can ascertain this since we visited the webpage. So:

In [111]:
last_names = transactions_w_nan['representative'].str.replace(' MD, FACS', '').str.split(' ').str[-1]
w_last_name = transactions_w_nan.assign(dc_code = last_names + transactions_w_nan['district'])
w_last_name.head()

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd,dc_code
0,2021,2021-10-04,2021-09-27,joint,BP,BP plc,purchase,"$1,001 - $15,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False,FoxxNC05
1,2021,2021-10-04,2021-09-13,joint,XOM,Exxon Mobil Corporation,purchase,"$1,001 - $15,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False,FoxxNC05
2,2021,2021-10-04,2021-09-10,joint,ILPT,Industrial Logistics Properties Trust - Common...,purchase,"$15,001 - $50,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False,FoxxNC05
3,2021,2021-10-04,2021-09-28,joint,PM,Phillip Morris International Inc,purchase,"$15,001 - $50,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False,FoxxNC05
4,2021,2021-10-04,2021-09-17,self,BLK,BlackRock Inc,sale_partial,"$1,001 - $15,000",Hon. Alan S. Lowenthal,CA47,https://disclosures-clerk.house.gov/public_dis...,False,LowenthalCA47


In [112]:
historical_members = pd.concat([historical_members_1, historical_members_2, historical_members_3])
last_name = historical_members['name'].str.split(',').str[0] 
w_at_large = historical_members['district'].mask(historical_members['district'].str.len() > 2, '00')
dc_code = historical_members['state'].replace(abbr_dct) + w_at_large.str.zfill(2)
w_code = historical_members.assign(dc_code=last_name + dc_code)
w_code = w_code[['party', 'dc_code']]
w_code.head()

,party,dc_code
0,Republican,AbrahamLA05
2,Democratic,AdamsNC12
4,Republican,AderholtAL04
6,Democratic,AguilarCA31
8,Republican,AllenGA12


In [113]:
w_party = pd.merge(w_last_name, w_code, how='left', on='dc_code')
w_party.head(1)

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd,dc_code,party
0,2021,2021-10-04,2021-09-27,joint,BP,BP plc,purchase,"$1,001 - $15,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False,FoxxNC05,Republican


In [117]:
set(w_last_name['dc_code']).difference(set(w_code['dc_code']))

{'ArenholzIA01', 'NortonDC00', 'SanchezCA38', 'SchultzFL23', 'SessionsTX32'}

In [119]:
w_code['dc_code'][w_code['dc_code'].str.contains('Hinson')]

418    HinsonIA01
Name: dc_code, dtype: object

In [121]:
w_code['dc_code'][w_code['dc_code'].str.contains('DC')]

198    Delegate NortonDC00
Name: dc_code, dtype: object

In [122]:
w_code['dc_code'][w_code['dc_code'].str.contains('CA38')]

412    SánchezCA38
Name: dc_code, dtype: object

In [123]:
w_code['dc_code'][w_code['dc_code'].str.contains('FL23')]

484    Wasserman SchultzFL23
Name: dc_code, dtype: object

In [124]:
w_code['dc_code'][w_code['dc_code'].str.contains('TX32')]

10    AllredTX32
Name: dc_code, dtype: object

In [110]:
w_party[w_party.isna()]

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd,dc_code_x,party,dc_code_y
0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12346,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
transactions_w_nan.iloc[0]

disclosure_year                                                        2021
disclosure_date                                         2021-10-04 00:00:00
transaction_date                                        2021-09-27 00:00:00
owner                                                                 joint
ticker                                                                   BP
asset_description                                                    BP plc
type                                                               purchase
amount                                                     $1,001 - $15,000
representative                                           Hon. Virginia Foxx
district                                                               NC05
ptr_link                  https://disclosures-clerk.house.gov/public_dis...
cap_gains_over_200_usd                                                False
Name: 0, dtype: object

This seems to have worked. For a quick eye test, let's sample 10 random people, and check to see if their party matches up with what it is on the official website.

In [99]:
w_party[['representative', 'party']].sample(10)

,representative,party
4002,Hon. Donald Sternoff Beyer,Democratic
6622,Hon. Mark Green,Republican
3201,Hon. Susie Lee,Democratic
10551,Hon. John Rutherford,Republican
2939,Hon. John A. Yarmuth,Democratic
11922,Hon. Dean Phillips,Democratic
3040,Hon. Kevin Hern,Republican
1572,"Hon. Charles J. ""Chuck"" Fleischmann",Republican
6523,Hon. Joe Courtney,Democratic
8439,Hon. Rohit Khanna,Democratic


### Assessment of Missingness

In [18]:
# TODO

### Hypothesis Test / Permutation Test

In [19]:
# TODO